In [3]:
from pynq import Overlay
from pynq import Xlnk
import numpy as np

In [4]:
# Load the overlay
overlay = Overlay("current_pool2D.bit")
pool_2D = overlay.max_pool2_0
overlay.is_loaded()

True

In [5]:
xlnk = Xlnk()
# Allocate memory for input data

ch_in = 3
h = 6
w = 6

h_o=h//2    # //为取整除法，确保h_o为整数，确保w_o为整数
w_o=w//2  

 
feature_in = xlnk.cma_array(shape=(ch_in, h, w), cacheable=0, dtype=np.float32)         # shape中的参数要确保是整数
feature_out = xlnk.cma_array(shape=(ch_in, h_o, w_o), dtype=np.float32)     



feature_in[:] = np.array([
    [[0, 1, 1, 4, 5, 1], 
     [1, 1, 1, 6, 8, 3], 
     [1, 1, 1, 8, 2, 6],
     [1, 1, 1, 8, 1, 3],
     [1, 2, 1, 4, 3, 7],
     [1, 1, 1, 8, 9, 6]],
    
    [[0, 1, 1, 4, 5, 1], 
     [1, 1, 1, 6, 8, 3], 
     [1, 1, 1, 8, 2, 6],
     [1, 1, 1, 8, 1, 3],
     [1, 2, 1, 4, 3, 7],
     [1, 1, 1, 8, 9, 6]],
    
    [[0, 1, 1, 4, 5, 1], 
     [1, 1, 1, 6, 8, 3], 
     [1, 1, 1000, 8, 2, 6],
     [1, 1, 1, 8, 1, 3],
     [1, 2, 1, 4, 3, 7],
     [1, 999, 1, 8, 9, 666]]
], dtype=np.float32)

In [9]:
def Run_maxpool2D(pool, chin, h, w, feature_in, feature_out):
    pool.write(0x10,h)
    pool.write(0x18,w)
    pool.write(0x20,ch_in)
    pool.write(0x28,feature_in.physical_address)
    pool.write(0x30,feature_out.physical_address)
    pool.write(0, (pool.read(0)&0x80)|0x01 ) #start conv IP
    while True:
        if pool.read(0x00) & 0x2:  # 检查 ap_done 位
            print('done')
            break
            

In [10]:
Run_maxpool2D(pool_2D, ch_in, h, w, feature_in, feature_out)
print("maxpool_2D result:")
print(feature_out)

done
maxpool_2D result:
[[[    1.     6.     8.]
  [    1.     8.     6.]
  [    2.     8.     9.]]

 [[    1.     6.     8.]
  [    1.     8.     6.]
  [    2.     8.     9.]]

 [[    1.     6.     8.]
  [    1.  1000.     6.]
  [  999.     8.   666.]]]
